## 1. Importar las librerías necesarias

Importamos las librerías principales como *pandas* para el manejo de datos, *matplotlib* y *seaborn* para las visualizaciones.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import HTML

## 2. Cargar los datasets

Se cargan los datasets books.csv (libros), ratings.csv (calificaciones de los usuarios) y to_read.csv (libros que los usuarios marcan para leer).

In [2]:
df_books = pd.read_csv("dataset/books.csv")
df_ratings = pd.read_csv("dataset/ratings.csv")
df_to_read = pd.read_csv("dataset/to_read.csv")
tags_df = pd.read_csv("dataset/tags.csv")
book_tags_df = pd.read_csv("dataset/book_tags.csv")

## 3. Explorar la Información de los Datasets

Esta celda muestra la estructura de cada dataset, el número de columnas y si hay valores faltantes o problemas con los tipos de datos.

### Información del dataset de libros:

In [ ]:
df_books.info()

### Información sobre el dataset de ratings:

In [ ]:
df_ratings.info()

### Información sobre el dataset 'to_read':

In [ ]:
df_to_read.info()

### Información sobre el dataset de tags

In [ ]:
tags_df.info()

### Información sobre el dataset de book_tags

In [ ]:
book_tags_df.info()

## 4. Limpieza de los Datos

### Book Dataset
Se eliminan columnas que no son útiles para el sistema de recomendación y nuestra página. Se eliminan las filas que tienen valores nulos en original_publication_year porque no podemos determinar el año de publicación de esos libros y eso puede afectar a nuestro análisis. Se imputan valores faltantes en la columna language_code con la etiqueta "Other".

In [ ]:
# Nos quedamos con las columnas que nos interesan
df_books = df_books.drop(['id','best_book_id', 'work_id', 'books_count', 'isbn', 'isbn13', 'original_title', 'work_ratings_count', 'work_text_reviews_count', 'ratings_1', 'ratings_2', 'ratings_3', 'ratings_4', 'ratings_5'], axis=1)

# Eliminamos los libros duplicados y cuyos valores de 'original_publication_year' sean nulos
df_books.drop_duplicates(subset='title', keep='first', inplace=True)
df_books.dropna(subset='original_publication_year',inplace=True)

# Rellenamos los valores nulos de 'language_code' con 'Other'
df_books.fillna({'language_code':'Other'}, inplace=True)

# Visualizamos los primeros libros
df_books.sort_values('book_id', ascending=True).head(10)


### Rating Dataset
Se eliminan las valoraciones de usuarios a libros duplicadas. Se eliminan las valoraciones de los libros que tras la limpieza de df_books no se encuentran en este.

In [ ]:
# Eliminamos los ratings duplicados
df_ratings.drop_duplicates(subset = ['user_id','book_id'], keep = False, inplace= True)
print(df_ratings.shape)

# Eliminamos los ratings cuyos libros no estén en el dataset de libros
df_ratings = df_ratings[df_ratings['book_id'].isin(df_books['book_id'])]
print(df_ratings.shape)

# Visualizamos los primeros ratings
df_ratings.head()

### To Read Dataset
Se eliminan los duplicados entre los pares user_id y book_id. Se eliminan los libros en to_read que tras la limpieza de df_books no están disponibles

In [ ]:
# Eliminamos los to read duplicados
df_to_read.drop_duplicates(subset = ['user_id','book_id'], keep = False, inplace= True)

# Eliminamos los libros en to read cuyos libros no estén en el dataset de libros
df_to_read = df_to_read[df_to_read['book_id'].isin(df_books['book_id'])]

# Visualizamos los primeros libros en to read
df_to_read.head()


### Tags Dataset
Eliminamos los tags duplicados del dataset de tags para su posterior unión con el DataFrame que asocia books con tags.

In [ ]:
# Eliminamos los tags duplicados
tags_df = tags_df.drop_duplicates(subset='tag_id', keep='first')

# Visualizamos los primeros tags
tags_df.head()


### Book-Tags Dataset
Realizamos ciertas transformaciones (Renombrar columnas, eliminar libros no usados, tags no identificados...) del DataFrame para poder unirlo sin problema al otro DataFrame de tags y así relacionar el nombre de las etiquetas con los libros. 

In [ ]:
# Renombramos la columna goodreads_book_id a book_id para mayor claridad
book_tags_df = book_tags_df.rename(columns={'goodreads_book_id': 'book_id'})

# Eliminamos la columna count
book_tags_df = book_tags_df.drop(['count'], axis=1)

# Eliminamos aquellos tags asociados a libros que no estén en el dataset de libros
book_tags_df = book_tags_df[book_tags_df['book_id'].isin(df_books['book_id'])]

# Eliminamos aquellos tags que no estén en el dataset de tags
book_tags_df = book_tags_df[book_tags_df['tag_id'].isin(tags_df['tag_id'])]

# Eliminamos los tags duplicados
book_tags_df = book_tags_df.drop_duplicates(subset=['book_id', 'tag_id'], keep='first')
print(book_tags_df.head())
# Unimos los libros con tags con los nombres de los tags
join_tags = pd.merge(tags_df ,book_tags_df, left_on='tag_id', right_on='tag_id', how='left').drop(['tag_id'], axis=1)

# Visualizamos los primeros tags con sus nombres y sus asociaciones
join_tags.head()

Buscamos y reunimos los géneros de libros en una lista. Filtramos todo nuestro DataFrame con tags y libros para quedarnos solo con aquellas etiquetas que hagan referencia a uno de esots géneros, para poder asociar a cualquier libro uno de estos.

In [ ]:
# Lista de géneros
genres=["art", "biography", "business", "chick lit", "childrens", "christian", "classics",
        "comics", "contemporary", "cookbooks", "crime", "fantasy", "fiction", 
        "graphic novels", "historical fiction", "history", "horror", "humor and comedy",
        "manga", "memoir", "music", "mystery", "nonfiction", "paranormal", "philosophy",
        "poetry", "psychology", "religion", "romance", "science", "science fiction",
        "self help", "suspense", "spirituality", "sports", "thriller", "travel", "young adult"]

# Filtramos para quedarnos con los tags que sean géneros
tag_genres = join_tags[join_tags['tag_name'].isin(genres)].reset_index(drop=True)


# Visualizamos el DataFrame anterior pero solo con los géneros
tag_genres.head()


## 5. Análisis Exploratorio de Datos (EDA)

In [14]:
# Función para obtener los géneros de un libro
def create_html(top_books):
  
  # Crear el HTML para visualizar los libros con imágenes y estilo
  html_content = '''
  <table style="border-collapse: collapse; width: 100%;">
    <thead>
      <tr>
        <th style="border: 1px solid #dddddd; padding: 8px;">Título</th>
        <th style="border: 1px solid #dddddd; padding: 8px;">Autor</th>
        <th style="border: 1px solid #dddddd; padding: 8px;">Valoración</th>
        <th style="border: 1px solid #dddddd; padding: 8px;">Imagen</th>
      </tr>
    </thead>
    <tbody>
  '''

  # Crear las filas con los datos del DataFrame
  for _, row in top_books.iterrows():
      html_content += f'''
      <tr>
        <td style="border: 1px solid #dddddd; padding: 8px;">{row['title']}</td>
        <td style="border: 1px solid #dddddd; padding: 8px;">{row['authors']}</td>
        <td style="border: 1px solid #dddddd; padding: 8px;">{row['average_rating']}</td>
        <td style="border: 1px solid #dddddd; padding: 8px;">
          <img src="{row['image_url']}" alt="Book image" style="width: 50px; height: 75px;">
        </td>
      </tr>
      '''

  # Cerrar la tabla
  html_content += '''
    </tbody>
  </table>
  '''

  # Mostrar el contenido HTML con estilo
  return html_content


### Mostrar libros con mejores valoraciones

In [ ]:
# Obtener los 5 libros con mayor valoración
top_books = df_books.nlargest(5, 'average_rating')[['title', 'authors', 'average_rating', 'image_url']]

# Mostrar el contenido en HTML
HTML(create_html(top_books))

### Mostrar libros más populares
Elegimos los libros más populares basándonos en los libros con más valoraciones

In [ ]:
# Obtener los 5 libros con mayor número de valoraciones
popular_books = df_books.nlargest(5, 'ratings_count')[['title', 'authors', 'average_rating', 'image_url']]

# Mostrar el contenido en HTML
HTML(create_html(popular_books))

### Matriz de correlación del dataset de libros
Mediante esta podemos ver que características númericas influyen en la valoración de nuestros usuarios y asi poder centrarnos en esta a la hora de realizar nuestro sistema

In [ ]:
df_books.info()

In [ ]:
df = df_books.copy()
df = df.drop(['title', 'authors', 'language_code', 'image_url','small_image_url'], axis=1)


# Generar el gráfico de correlación entre las características de los libros
plt.figure(figsize=(10, 8))
sns.heatmap(df.corr(), annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Relación entre las características númericas de los libros')
plt.show()


### Autores con más libros

In [ ]:
# Autores con más número de libros
books_autor = df_books['authors'].value_counts().head(5)

# Visualizamos los autores con más libros
plt.figure(figsize=(8,6))
books_autor.plot(kind='bar', color='salmon', edgecolor='black')
plt.title('Autores con más número de libros')
plt.xlabel('Autores')
plt.ylabel('Número de libros')
plt.show()

### Autores con mejores valoraciones

In [ ]:

# Agrupar por autor y calcular el número de libros y la media de valoraciones
author_stats = df_books.groupby('authors').agg(
    num_books=('book_id', 'count'),
    avg_rating=('average_rating', 'mean')
).reset_index()

# Filtrar autores con al menos un número mínimo de libros (e.g., 3 libros)
min_books = 10
top_authors = author_stats[author_stats['num_books'] >= min_books]

# Ordenar por la media de valoraciones
top_authors = top_authors.sort_values(by='avg_rating', ascending=False).head(10)

# Graficar los autores con mejor nota media y al menos 3 libros
plt.figure(figsize=(10,6))
plt.barh(top_authors['authors'], top_authors['avg_rating'], color='skyblue', edgecolor='black')
plt.title('Mejores autores por número de libros y valoración media')
plt.xlabel('Valoración media')
plt.ylabel('Autores')
plt.gca().invert_yaxis()  # Invertir el eje Y para que el mejor esté arriba
plt.grid(True)
plt.tight_layout()
plt.show()

### Distribución de la nota media

In [ ]:
# Graficar la distribución de las valoraciones promedio
plt.figure(figsize=(8,6))
plt.hist(df_books['average_rating'], bins=10, color='skyblue', edgecolor='black')
plt.title('Distribución de las valoraciones promedio')
plt.xlabel('Average Rating')
plt.ylabel('Frecuencia')
plt.grid(True)
plt.show()

### Relación entre Número de valoraciones y valoración promedio

In [ ]:
# Graficar la relación entre el número de valoraciones y la valoración promedio

plt.figure(figsize=(10,6))

# Graficar con escala logarítmica en el eje de valoraciones y transparencia en los puntos
plt.scatter(df_books['ratings_count'], df_books['average_rating'], color='green', alpha=0.5, edgecolor='black')

# Escala logarítmica en el eje X para mejor distribución de los datos
plt.xscale('log')

plt.title('Relación: número de valoraciones y valoración promedio')
plt.xlabel('Número de valoraciones (escala logarítmica)')
plt.ylabel('Valoración promedio')

plt.grid(True)
plt.tight_layout()
plt.show()

### Géneros con más libros

In [23]:
# Contar el número de libros por género
tag_genres_count = tag_genres.groupby('tag_name').count()
tag_genres_count.sort_values(by='book_id', ascending=False, inplace=True)
tag_genres_count = tag_genres_count.rename(columns={'book_id': 'num_books'})


In [ ]:

# Graficar los géneros más comunes
plt.figure(figsize=(40,20))
tag_genres_count.plot(kind='bar', color='skyblue', edgecolor='black')
plt.title('Número de libros por género')
plt.xlabel('Género')
plt.ylabel('Número de libros')
plt.tight_layout()
plt.show()

### Libros más añadidos a "To read"

In [ ]:
# Unir los libros con los libros en to read
book_to_read = df_to_read.merge(df_books, on='book_id')

# Libros más añadidos a las listas de "leer más tarde"
books_most_added = book_to_read['title'].value_counts().head(10)


# Gráfico de libros más añadidos
plt.figure(figsize=(10,6))
books_most_added.plot(kind='barh', color='skyblue')
plt.title('Libros más añadidos a las listas de "leer más tarde"')
plt.xlabel('Veces añadido')
plt.gca().invert_yaxis()  # Invertir el eje para que el más añadido esté arriba
plt.tight_layout()
plt.grid(True)
plt.show()

### Autores cuyos libros más aparecen en listas "To Read"

In [ ]:
# Autores más añadidos a las listas de "leer más tarde"
authors_most_added = book_to_read['authors'].value_counts().head(10)

# Gráfico de autores más añadidos
plt.figure(figsize=(10,6))
authors_most_added.plot(kind='barh', color='lightgreen')
plt.title('Autores más añadidos a las listas de "leer más tarde"')
plt.xlabel('Veces añadido')
plt.gca().invert_yaxis()  # Invertir el eje para que el más añadido esté arriba
plt.tight_layout()
plt.show()

### Distribución de libros en "To Read" por año de publicación

In [ ]:
# Filtrar los datos para los últimos 100 años
filtered_last_100_years = book_to_read[book_to_read['original_publication_year'] >= (2024 - 100)]

# Crear el histograma para los últimos 100 años
plt.figure(figsize=(10,6))
plt.hist(filtered_last_100_years['original_publication_year'].dropna().astype(int), bins=40, color='lightcoral', edgecolor='black')
plt.title('Distribución de libros en listas de "leer más tarde" por año de publicación')
plt.xlabel('Año de publicación')
plt.ylabel('Número de libros añadidos')
plt.grid(True)

plt.tight_layout()
plt.show()

### Guardar Datasets Preprocesados
Tras explorar los datasets, ver su información y que características nos pueden interesar a la hora de realizar nuestro sistema de recomendación, procedemos a guardarlos.

In [ ]:
df_books.to_csv('clean_datasets/books.csv', index=False)
df_ratings.to_csv('clean_datasets/ratings.csv', index=False)
df_to_read.to_csv('clean_datasets/to_read.csv', index=False)
tag_genres.to_csv('clean_datasets/tags.csv', index=False)